In [ ]:
!pip install cryptography

     |████████████████████████████████| 3.6 MB 8.8 MB/s 


# **Utlity Functions**

In [ ]:
# Convert to string in ascii from numeric representation of triplets
def convert(a) :
    l = len(a)
    s = ""
    
    idx = 0
    while idx < l :
        temp = a[idx:idx+3]
        s = s + chr(int(temp))
        idx = idx + 3
        
    return s

# Creating a random salt of length n
import string
import random

def get_random_salt(n) :
  
    salt=""
    for i in range(n):
        salt=salt+random.choice(string.ascii_letters)

    return salt

# Get a random encryption key
from cryptography.fernet import Fernet

def get_random_encryptkey() :

    encrypt_key = Fernet.generate_key()
    # fernet = Fernet(key)
    return encrypt_key


In [ ]:
# Take XOR of string a and string b
def XOR1(a, b) :
    res = ""
    i = 0
    j = 0
    
    while (i<len(a) and j<len(b)) :
        
        temp = str(ord(a[i]) ^ ord(b[j])) 
        
        if len(temp) == 1 :
            temp = "00" + temp
        elif len(temp) == 2 :
            temp = "0" + temp
        
        res += temp
        i = i + 1
        j = j + 1
    
    while i<len(a) :
        
        temp = str(ord(a[i])) 
        
        if len(temp) == 1 :
            temp = "00" + temp
        elif len(temp) == 2 :
            temp = "0" + temp
        
        res += temp
        i = i + 1
        
    while j<len(b) :
        
        temp = str(ord(b[j])) 
        
        if len(temp) == 1 :
            temp = "00" + temp
        elif len(temp) == 2 :
            temp = "0" + temp
        
        res += temp
        j = j + 1
        
    return res      


# **Connecting with MySQL**

In [ ]:
pip install mysql-connector-python-rf

     |████████████████████████████████| 11.9 MB 6.0 MB/s 
  Created wheel for mysql-connector-python-rf: filename=mysql_connector_python_rf-2.2.2-cp37-cp37m-linux_x86_64.whl size=249476 sha256=f857227b978c96db312dc95e0915a3f1ad48f3157fd1f2a0310639213074752f
  Stored in directory: /root/.cache/pip/wheels/68/59/cf/3b03557b26b4c75af3788a553e0ff9cf0b37a22d0c9cb01979
Successfully built mysql-connector-python-rf


In [ ]:
# install, set connection
!apt-get install mysql-server > /dev/null
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"
!pip -q install PyMySQL
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql mysql+pymysql://root:root@/
# query using %sql or %%sql
df = %sql SELECT Host, User, authentication_string FROM mysql.user
df

 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.
     |████████████████████████████████| 43 kB 1.3 MB/s 
 * mysql+pymysql://root:***@/


,Host,User,authentication_string
0,localhost,root,*81F5E21E35407D884A6CD4A731AEBFB6AF209E1B
1,localhost,mysql.session,*THISISNOTAVALIDPASSWORDTHATCANBEUSEDHERE
2,localhost,mysql.sys,*THISISNOTAVALIDPASSWORDTHATCANBEUSEDHERE
3,localhost,debian-sys-maint,*5BEBF7FE37CD488CD634F9C542C1C481E0BEFB3C


In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root"
)

print(mydb)

# **Part 1 : Store Data in Database**

In [ ]:
mycursor = mydb.cursor()

In [ ]:
mycursor.execute("DROP DATABASE btp")

In [ ]:
mycursor.execute("CREATE DATABASE btp")

In [ ]:
mycursor.execute("SHOW DATABASES")
for x in mycursor:
  print(x)

('information_schema',)
('btp',)
('mysql',)
('performance_schema',)
('sys',)


In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root",
  database='btp'
)

mycursor = mydb.cursor()

In [ ]:
table_name = "customers"
columns = ['id', 'name', 'city', 'wallet_balance']
data = [ ('1', 'Mangesh', 'Umarkhed', '100'),
        ('2', 'Puspesh', 'Patna', '200'),
        ('3', 'Ritwiz', 'Delhi', '150'),
        ('4', 'Pranay', 'Delhi', '190'),
        ('5', 'Harsh', 'Agra', '120') ]

In [ ]:
def store_data(table_name, columns, data) :

    # create table query, first column shoud be id/primary key
    create_query = "CREATE TABLE " + table_name + " (" + columns[0] + " VARCHAR(511) PRIMARY KEY"

    for idx in range(1,len(columns)):
        create_query += ", " + columns[idx] + " VARCHAR(511)"
    
    create_query += ")"

    mycursor.execute(create_query)

    # insert data query 
    insert_query = "INSERT INTO " + table_name + " (" + columns[0]

    for idx in range(1,len(columns)):
        insert_query += ", " + columns[idx]
    
    insert_query += ") VALUES (" + "%s"

    for idx in range(1,len(columns)):
        insert_query += ", %s"

    insert_query += ")"

    id_and_salts = []
    encrypt_key = get_random_encryptkey();
    fernet = Fernet(encrypt_key)
    encrypted_data = []
    
    for idx in range(len(data)):
        encrypted_data.append(list(data[idx]))

    for row in range(len(data)) :
      
        encrypted_id = fernet.encrypt(data[row][0].encode('utf-8'))
        salt = get_random_salt(len(encrypted_id))
        
        id_and_salts.append(encrypted_id)
        id_and_salts.append(salt)

        encrypted_data[row][0] = XOR1(str(encrypted_id),salt)

        for col in range(1,len(data[0])) :

            encrypted_data[row][col] = XOR1(str(fernet.encrypt(data[row][col].encode('utf-8'))), salt)
                
    mycursor.executemany(insert_query, encrypted_data)
    mydb.commit()

    print("Encryption Key :", encrypt_key)

    idx = 0
    print("Encrypted Primary Key and Salt :")
    while idx < len(id_and_salts) :
        if idx == 0:
            print(str(id_and_salts[idx]) + "," + id_and_salts[idx+1])
        else :
            print("," + str(id_and_salts[idx]) + "," + id_and_salts[idx+1])
        idx = idx + 2


In [ ]:
store_data(table_name, columns, data)

Encryption Key : b'yJPjPfhNXxNDTrRpjeO1MxR0Qdx7WunT6u96HgjU7YI='
Encrypted Primary Key and Salt :
b'gAAAAABhpfRNgw9c3epnjiG_S4OrmQkwdS3y5gC-Rmq6QMsPjE0J_RH6Sdvfja6OeD3btdh0u50YYDA-mFtEJCvi33CEgYveVQ==',QQniafuwaJzsMgYRvAwtZtuEdqnBaioFDnogobLzIzApdjjjEdGWGjBIBUKkFoobNunERtsOfrsXmbkNRkcYUaTkECMVwliTowVA
,b'gAAAAABhpfRN6J5XvErrUMfyYCbrxmGsNOJQvFr4Jn5Po-eAvMnFrqugz0cttMhZIsqOCOxvglF_nOwdWno-I-9_oqtVFCIHEQ==',bySrmKndMWdPAvZdhyyQvwrQkOrGnHxrpbqovDraXgiQEwEaxrRFGFlsYMuRruZfBBtMQDNKPqHjZyMluWPbXTgFvcZMJdOFbItm
,b'gAAAAABhpfRNhAqpi_PG2pj8q7jTdaao4G60xtwVbWn1Uthzn0FBziF1oMhw_BuOvCQdko3oZCHGz_aJ7FjV3A7juUXgU392hQ==',TWmduBnPEeMPbAPsYWoUxCJnMfdYlrZIfSTKgmwogSLVcarSJfOrgMgmbmopkLEAzHMPBtifrvIFHaVYovYwjNLOmLNtaGygGDgz
,b'gAAAAABhpfRNfVHie8b4Az2SjTBSk3OkXaQ0c39Y75URirG1YrQXBS3tgqDRDvEzOD5LRrydSCJ3lST6omj5x8Xo3lITG7QVVw==',OYsswyMmgeKSrvuBcleOvnUqiVIFTsPkgsQvEunRemSSIGWyPRYFzZtnNcgjauNckjGREcfwCOueeTcUJszWztdrlASbowKdviLm
,b'gAAAAABhpfRNodZbcMjaW14uWSjC5rhXR8Ph6fZrTcnRm7cRflpB_L65uR_zHPPf3H-7RXEdXWdI

In [ ]:
mycursor.execute("SHOW TABLES")
for x in mycursor:
  print(x)

('customers',)


In [ ]:
mycursor.execute("SELECT * from customers")
for x in mycursor:
  print(x)

('005065048010055027055027041026032003055004018043024059055016060097049010007030044030091055052042056016020088127093015037051042047079043106014092009047047005009018063026007045037079083000037125054005037034049008024016033014077037003062029024072068101025024051053062005054054031014059081109096097033048061061039', '005065048010055027055027041026032003055004050025064027044008040005035010036103033088091048060059035015046031028051052081105025023061089040054087076027079035060014026014001123033012050062015028081066088045036034066052068124058013028063035015060077099041048007050028020036030002064052046047001060004022061061039', '005065048010055027055027041026032003055004060046003052046029056058015057035003030006022056063004063031011009059043109000056122097027085106018059047049047056041102089031020119013043083039051025030032088031030045022010032063072051054020016113009032101047040048019026123023034025060057035019034051002048061061039', '00506504801005502705502704102603200305500402505106901004705

# **PART 2 : Making Query from MySQL**

In [ ]:

# Get from result of store_data 
key1=b'yJPjPfhNXxNDTrRpjeO1MxR0Qdx7WunT6u96HgjU7YI='
EncPrimary_key="b'gAAAAABhpfRNodZbcMjaW14uWSjC5rhXR8Ph6fZrTcnRm7cRflpB_L65uR_zHPPf3H-7RXEdXWdIqP3o_kbKRymBpH6V9np_wQ=='"

# Get from Blockchain
salt_str="OqvaDNhgxIwyQhlDNHwmaFvyScQuPoyRnZiYYKdaNbJwPEVhcuPYbUpTWRtciMYrfuCIrTxRGXeZjcryLafLYsStILtUnmchanLG"

Idd=(XOR1(salt_str,EncPrimary_key), )

query = "SELECT name from customers where id= %s"
mycursor.execute(query,Idd)
res=""
for x in mycursor:
  #print(x)
  res=x

Encrypted_res=XOR1(convert(res[0]),salt_str)
#print(Encrypted_res)
#print(type(res[0]))
fernet1=Fernet(key1)
temp=convert(Encrypted_res)
print(temp)
s1=temp[1:len(temp)]
decMessage = fernet1.decrypt(s1.encode('utf-8')).decode()

print("\n\nDecrypted string: ", decMessage)

b'gAAAAABhpfRNgoSSigRfeQkaPpoEJkQGcjvIsPLLAUR6Zym9dMistlXFuqJ-djaha1yk_-gxGv_PQ4-kpPI_OnHEv2yPh0yEbg=='


Decrypted string:  Pranay
